# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [16]:
# import libraries
import numpy as np
import pandas as pd
import re
from sqlalchemy import create_engine
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
from sklearn.metrics import classification_report
from time import time

nltk.download(['punkt', 'wordnet', 'stopwords'])


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dina\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dina\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dina\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql('data', engine)

df = pd.read_sql('data', engine)  # Load data from database to pandas DataFrame
categories = df.select_dtypes(include=['int64'])  # Select only int64 datatypes
categories = categories.drop('id', axis=1)  # Drop id column as irrelevant
X = df['message'].values
y = categories.values

X.shape


(26027,)

In [4]:
type(X)

numpy.ndarray

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower().strip())  # Normalize text and remove white space
    words = word_tokenize(text)  # Tokenize text
    words = [w for w in words if w not in stopwords.words("english")]  # Remove stop words
    words = [WordNetLemmatizer().lemmatize(w) for w in words]  # Lemmatize
    words = [WordNetLemmatizer().lemmatize(w, pos='v') for w in words]  # Lemmatize verbs by specifying pos
    
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
classifier = MultiOutputClassifier(RandomForestClassifier(n_jobs=-1))

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', classifier)
    ])

print(pipeline)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000011B2B23FF70>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))])


In [11]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000011B2B23FF70>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000011B2B23FF70>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__subl

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
# train classifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)

yhat = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
category_names = list(categories.columns)

for i in range(len(category_names)):
    print("Category: {}\n {}\n {}\n______".format(
        category_names[i],
        classification_report(y_test[:,i], yhat[:,i]),
        confusion_matrix(y_test[:,i], yhat[:,i])
    ))



Category: related
               precision    recall  f1-score   support

           0       0.74      0.38      0.50      1249
           1       0.83      0.96      0.89      3957

    accuracy                           0.82      5206
   macro avg       0.79      0.67      0.70      5206
weighted avg       0.81      0.82      0.80      5206

 [[ 476  773]
 [ 167 3790]]
______
Category: request
               precision    recall  f1-score   support

           0       0.90      0.98      0.94      4301
           1       0.82      0.51      0.63       905

    accuracy                           0.90      5206
   macro avg       0.86      0.74      0.78      5206
weighted avg       0.89      0.90      0.89      5206

 [[4203   98]
 [ 445  460]]
______
Category: offer
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      5180
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5206
   

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Category: water
               precision    recall  f1-score   support

           0       0.96      1.00      0.98      4885
           1       0.91      0.42      0.58       321

    accuracy                           0.96      5206
   macro avg       0.94      0.71      0.78      5206
weighted avg       0.96      0.96      0.96      5206

 [[4872   13]
 [ 185  136]]
______
Category: food
               precision    recall  f1-score   support

           0       0.95      0.98      0.97      4611
           1       0.84      0.63      0.72       595

    accuracy                           0.94      5206
   macro avg       0.90      0.81      0.84      5206
weighted avg       0.94      0.94      0.94      5206

 [[4541   70]
 [ 221  374]]
______
Category: shelter
               precision    recall  f1-score   support

           0       0.94      0.99      0.96      4728
           1       0.83      0.36      0.50       478

    accuracy                           0.93      5206
   ma

______
Category: earthquake
               precision    recall  f1-score   support

           0       0.98      0.99      0.98      4722
           1       0.89      0.79      0.84       484

    accuracy                           0.97      5206
   macro avg       0.94      0.89      0.91      5206
weighted avg       0.97      0.97      0.97      5206

 [[4676   46]
 [ 101  383]]
______
Category: cold
               precision    recall  f1-score   support

           0       0.98      1.00      0.99      5089
           1       0.88      0.06      0.11       117

    accuracy                           0.98      5206
   macro avg       0.93      0.53      0.55      5206
weighted avg       0.98      0.98      0.97      5206

 [[5088    1]
 [ 110    7]]
______
Category: other_weather
               precision    recall  f1-score   support

           0       0.95      1.00      0.97      4945
           1       0.70      0.03      0.05       261

    accuracy                           0.9

In [24]:
for i in range(len(category_names)):
    print("Category: {}\n Accuracy: {:.3f}\t Precision: {:.3f}\t Recall: {:.3f} \n\n".format(
        category_names[i],
        accuracy_score(y_test[:,i], yhat[:,i]),
        precision_score(y_test[:,i], yhat[:,i]),
        recall_score(y_test[:,i], yhat[:,i])
        )
    )

Category: related
 Accuracy: 0.819	 Precision: 0.831	 Recall: 0.958 


Category: request
 Accuracy: 0.896	 Precision: 0.824	 Recall: 0.508 


Category: offer
 Accuracy: 0.995	 Precision: 0.000	 Recall: 0.000 


Category: aid_related
 Accuracy: 0.790	 Precision: 0.769	 Recall: 0.706 


Category: medical_help
 Accuracy: 0.925	 Precision: 0.725	 Recall: 0.090 


Category: medical_products
 Accuracy: 0.954	 Precision: 0.774	 Recall: 0.093 


Category: search_and_rescue
 Accuracy: 0.975	 Precision: 0.750	 Recall: 0.044 


Category: security
 Accuracy: 0.979	 Precision: 1.000	 Recall: 0.009 


Category: military
 Accuracy: 0.967	 Precision: 0.857	 Recall: 0.034 


Category: child_alone
 Accuracy: 1.000	 Precision: 0.000	 Recall: 0.000 


Category: water
 Accuracy: 0.962	 Precision: 0.913	 Recall: 0.424 


Category: food
 Accuracy: 0.944	 Precision: 0.842	 Recall: 0.629 


Category: shelter
 Accuracy: 0.934	 Precision: 0.834	 Recall: 0.358 


Category: clothing
 Accuracy: 0.986	 Precision: 0.

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 0.75, 1.0),
        'vect__max_features': (None, 5000, 10000),
        'tfidf__use_idf': (True, False),
        'tfidf__norm': ('l1', 'l2'),
        'clf__estimator__n_estimators': [50, 100, 200],
        'clf__estimator__min_samples_split': [2, 4, 6],
        'clf__estimator__max_depth': [80, 90, 100, 110],
        'clf__estimator__n_estimators': [100, 200, 300, 1000]
}


cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1)
    

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.